In [1]:
import sys
sys.path.append('..')

import utils, selex_dca, indep_sites
import adabmDCA
import selex_distribution, energy_models, tree, data_loading, training, callback, sampling

import torch
import numpy as np
from utils import one_hot
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
from adabmDCA.sampling import get_sampler
import random

/home/scrotti/Aptamer2025py/selex_dca.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
%load_ext line_profiler

In [6]:
L = 45
q = 4
M = 10**4

x_ = torch.randint(q, (M, L))
x = one_hot(x_)

In [7]:
def mlp(n_hidden):
    return torch.nn.Sequential(
        torch.nn.Linear(L*q, n_hidden),
        torch.nn.ReLU(),
        torch.nn.Linear(n_hidden, 1)
    )

def linear(dtype, device):
    return torch.nn.Linear(L*q, 1).to(dtype=dtype, device=device)

In [16]:
h = torch.randn(L, q)
J_ = torch.randn(L*q, L*q) / torch.sqrt(torch.Tensor([L]))
J_ = J_ + J_.t()
J = J_.reshape(L, q, L, q)
potts = energy_models.Potts(J, h)

n_hidden = 50
nn = energy_models.GenericEnergyModel(mlp(n_hidden))

In [17]:
%timeit potts.compute_energy(x)

2.68 ms ± 96.1 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
